In [1]:
import numpy
import numpy.random
import pandas
import vtreat

In [2]:
numpy.random.seed(2019)

In [3]:
n_rows = 1000
y_levels = ['a', 'b', 'c']
d = pandas.DataFrame({'y': numpy.random.choice(y_levels, size=n_rows)})
# signal variables, correlated with y-levels
for i in range(2):
    vmap = {yl: numpy.random.normal(size=1)[0] for yl in y_levels}
    d['var_n_' + str(i)] = [vmap[li] + numpy.random.normal(size=1)[0] for li in d['y']]
for i in range(2):
    col = numpy.random.choice(y_levels, size=n_rows)
    col = [col[i] if numpy.random.uniform(size=1)[0]<=0.8 else d['y'][i] for i in range(n_rows)]
    d['var_c_' + str(i)] = col
# noise variables, uncorrelated with y-levels
for i in range(2):
    d['noise_n_' + str(i)] = [numpy.random.normal(size=1)[0] + numpy.random.normal(size=1)[0] for li in d['y']]
for i in range(2):
    d['noise_c_' + str(i)] = numpy.random.choice(y_levels, size=n_rows)
d.head()

,y,var_n_0,var_n_1,var_c_0,var_c_1,noise_n_0,noise_n_1,noise_c_0,noise_c_1
0,a,0.544891,1.595448,b,a,1.196747,-0.714955,a,a
1,c,-0.433273,0.778452,a,b,0.332786,-1.175452,a,b
2,b,-1.230834,0.859162,b,a,-1.454101,-3.079244,a,b
3,a,1.158161,-0.344363,c,b,-1.175055,-1.314302,a,b
4,c,-1.029690,0.789506,c,c,0.809634,0.951092,b,c


In [4]:
treatment = vtreat.MultinomialOutcomeTreatment(
    outcome_name='y')
cross_frame = treatment.fit_transform(d, d['y'])
cross_frame.head()

,y,var_n_0,var_n_1,var_c_0_logit_code_a,var_c_0_logit_code_c,var_c_0_logit_code_b,var_c_0_prevalence_code,var_c_0_lev_b,var_c_0_lev_a,var_c_0_lev_c,var_c_1_logit_code_a,var_c_1_logit_code_c,var_c_1_logit_code_b,var_c_1_prevalence_code,var_c_1_lev_a,var_c_1_lev_b,var_c_1_lev_c,noise_c_0_logit_code_a,noise_c_0_prevalence_code,noise_c_1_logit_code_b
0,a,0.544891,1.595448,-0.270671,-0.170676,0.335207,0.351,1,0,0,0.157770,-0.157605,-0.029251,0.345,1,0,0,-0.029763,0.334,-0.023318
1,c,-0.433273,0.778452,0.292508,-0.160168,-0.194231,0.327,0,1,0,-0.102289,-0.279495,0.317727,0.333,0,1,0,-0.010650,0.334,-0.006743
2,b,-1.230834,0.859162,-0.165349,-0.213881,0.305318,0.351,1,0,0,0.143579,-0.212065,0.031372,0.345,1,0,0,-0.010650,0.334,-0.006743
3,a,1.158161,-0.344363,-0.204105,0.306458,-0.246618,0.319,0,0,1,-0.102289,-0.279495,0.317727,0.333,0,1,0,-0.010650,0.334,-0.006743
4,c,-1.029690,0.789506,-0.170503,0.248976,-0.191327,0.319,0,0,1,-0.094303,0.343863,-0.439817,0.319,0,0,1,0.037777,0.341,0.020937


In [5]:
treatment.score_frame_

,variable,orig_variable,treatment,y_aware,has_range,PearsonR,significance,vcount,recommended,outcome_target
0,var_n_0,var_n_0,clean_copy,False,True,0.460713,1.079644e-53,4.0,True,a
1,var_n_1,var_n_1,clean_copy,False,True,-0.253237,4.259663e-16,4.0,True,a
2,noise_n_0,noise_n_0,clean_copy,False,True,0.003113,9.216881e-01,4.0,False,a
3,noise_n_1,noise_n_1,clean_copy,False,True,-0.030264,3.390479e-01,4.0,False,a
4,var_c_0_logit_code_a,var_c_0,logit_code,True,True,0.171036,5.268229e-08,12.0,True,a
5,var_c_0_logit_code_c,var_c_0,logit_code,True,True,-0.044372,1.608856e-01,12.0,False,a
6,var_c_0_logit_code_b,var_c_0,logit_code,True,True,-0.107514,6.603745e-04,12.0,False,a
7,var_c_0_prevalence_code,var_c_0,prevalence_code,False,True,-0.060871,5.431808e-02,4.0,True,a
8,var_c_0_lev_b,var_c_0,indicator_code,False,True,-0.100126,1.522917e-03,12.0,True,a
9,var_c_0_lev_a,var_c_0,indicator_code,False,True,0.184872,3.868879e-09,12.0,True,a
